# Imports

In [1]:
# !pip install modin[ray]
!pip install pandasql

     |████████████████████████████████| 761 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 10.5 MB/s 
     |████████████████████████████████| 52.7 MB 47 kB/s 
     |████████████████████████████████| 226 kB 34.6 MB/s 
     |████████████████████████████████| 4.1 MB 32.0 MB/s 
     |████████████████████████████████| 65 kB 2.9 MB/s 
     |████████████████████████████████| 128 kB 32.7 MB/s 
     |████████████████████████████████| 3.0 MB 30.6 MB/s 
     |████████████████████████████████| 82 kB 189 kB/s 
     |████████████████████████████████| 144 kB 33.9 MB/s 
     |████████████████████████████████| 1.1 MB 17.4 MB/s 
     |████████████████████████████████| 201 kB 21.9 MB/s 
     |████████████████████████████████| 271 kB 6.6 MB/s 
     |████████████████████████████████| 94 kB 1.5 MB/s 
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for gpustat: filename=gpustat-1.0.0b1-py3-none-any.whl size=15979 sha

In [2]:
import pandas as pd
import pandasql as ps
# import modin.pandas as pd # multithreaded version of pandas

# String url for each dataset

All data is from https://github.com/JeffSackmann/tennis_atp

In [ ]:
seed = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_"
match_strings = [str(year) + ".csv" for year in range(1968, 2023)]
match_urls = [seed + match_string for match_string in match_strings]

seed = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_doubles_"
match_strings = [str(year) + ".csv" for year in range(2000, 2021)]
match_double_urls = [seed + match_string for match_string in match_strings]

seed = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_futures_"
match_strings = [str(year) + ".csv" for year in range(1991, 2023)]
match_futures_urls = [seed + match_string for match_string in match_strings]

seed = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_qual_chall_"
match_strings = [str(year) + ".csv" for year in range(1978, 2023)]
match_qual_urls = [seed + match_string for match_string in match_strings]

players_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv"

seed = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_"
match_strings = ["%d0s.csv" % i for i in range(10)]
rankings_urls = [seed + match_string for match_string in match_strings]

ameteurs_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_amateur.csv"

# Joining all the dataframes

In [ ]:
# takes in list of csv urls and joins them all together
def join_dfs(urls):
  joined_df = pd.DataFrame()
  for df_url in urls:
    try:
      joined_df = pd.concat([joined_df, pd.read_csv(df_url)], axis=0)
    except:
      print("doesnt exist:", df_url)
  return joined_df

Joining each dataframe

In [ ]:
matches = join_dfs(match_urls)

In [ ]:
ameteurs = pd.read_csv(ameteurs_url)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
doubles = join_dfs(match_double_urls)

In [ ]:
futures = join_dfs(match_futures_urls)

In [ ]:
quals = join_dfs(match_qual_urls)

In [ ]:
rankings = join_dfs(rankings_urls)

doesnt exist: https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_30s.csv
doesnt exist: https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_40s.csv
doesnt exist: https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_50s.csv
doesnt exist: https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_60s.csv


In [ ]:
players = pd.read_csv(players_url)

# Organizing and saving CSV files for AWS

In [ ]:
players.to_csv("players.csv")

In [ ]:
rankings.to_csv("rankings.csv")

In [ ]:
a = """CREATE TABLE Matches (
	tourney_id VARCHAR(50),
tourney_name VARCHAR(25),
surface VARCHAR(10),
draw_size INT,
tourney_level VARCHAR(1),
tourney_date DATE,
match_num INT,
winner_id INT,
winner_seed INT,
winner_entry VARCHAR(10),
loser_id INT,
loser_seed INT,
loser_entry VARCHAR(10),
score VARCHAR(25),
best_of INT,
round VARCHAR(5),
minutes INT,
w_ace INT,
w_df INT,
w_svpt INT,
w_1stIn INT,
w_1stWon INT,
w_2ndWon INT,
w_SvGms INT,
w_bpSaved INT,
w_bpFaced INT,
l_ace INT,
l_df INT,
l_svpt INT,
l_1stIn INT,
l_1stWon INT,
l_2ndWon INT,
l_SvGms INT,
l_bpSaved INT,
l_bpFaced INT,
PRIMARY KEY (tourney_id, start_date, match_num),
FOREIGN KEY (tourney_id) REFERENCES Tournaments(tourney_id),
FOREIGN KEY (start_date) REFERENCES Tournaments(tourney_id),
FOREIGN KEY (winner_id) REFERENCES Players(player_id),
FOREIGN KEY (loser_id) REFERENCES Players(player_id)
);
""".split("\n")

b = [w.split(" ")[0] for w in a]
b = b[2:-7]

In [ ]:
matches_cleaned = matches[b]
matches_cleaned.to_csv("matches.csv")

In [ ]:
matches_cleaned.columns

Index(['tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'match_num', 'winner_id', 'winner_seed', 'winner_entry', 'loser_id',
       'loser_seed', 'loser_entry', 'score', 'best_of', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'],
      dtype='object')

In [ ]:
a = matches[["round", "tourney_name", "winner_id"]]
a[a["round"] == "F"]

,round,tourney_name,winner_id
30,F,Dublin,100084
37,F,Buenos Aires,100022
159,F,Gstaad,100055
190,F,Kitzbuhel,100050
221,F,Bournemouth,100016
...,...,...,...
625,F,Santiago,124079
652,F,Indian Wells Masters,126203
747,F,Miami Masters,207989
842,F,Houston,124187


In [ ]:
players.head(10)

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122,USA,NaN,Q54544
1,100002,Pancho,Segura,R,19210620,ECU,NaN,Q54581
2,100003,Frank,Sedgman,R,19271002,AUS,NaN,Q962049
3,100004,Giuseppe,Merlo,R,19271011,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509,USA,188.0,Q53554
5,100006,Grant,Golden,R,19290821,USA,175.0,Q3115390
6,100007,Abe,Segal,L,19301023,RSA,NaN,Q1258527
7,100008,Kurt,Nielsen,R,19301119,DEN,NaN,Q552261
8,100009,Istvan,Gulyas,R,19311014,HUN,NaN,Q51066
9,100010,Luis,Ayala,R,19320918,CHI,NaN,Q1275397


In [ ]:
rankings.head(10)

,ranking_date,rank,player,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0
2,20000110,3,101948,2419.0
3,20000110,4,103017,2184.0
4,20000110,5,102856,2169.0
5,20000110,6,102358,2107.0
6,20000110,7,102839,1966.0
7,20000110,8,101774,1929.0
8,20000110,9,102701,1846.0
9,20000110,10,101990,1739.0


In [ ]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
q = """SELECT ioc AS country, COUNT(*) AS num_champs 
FROM matches M 
     JOIN (SELECT player_id, ioc 
           FROM players) P ON M.winner_id = P.player_id 
GROUP BY country 
ORDER BY COUNT(*) DESC
"""

ps.sqldf(q, locals())

,country,num_champs
0,USA,37213
1,ESP,15256
2,AUS,13772
3,FRA,11549
4,GER,8885
...,...,...
124,UAE,2
125,GUM,1
126,MNE,1
127,RHO,1


# EDA

In [ ]:
def find_similar_cols(df1, df2):
  unique = []
  for col1 in df1.columns:
    for col2 in df2.columns:
      if col1 in unique:
        pass
      else:
        if col1 == col2:
          unique.append(col1)
  return unique

In [ ]:
find_similar_cols(matches, players)

[]

In [ ]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
matches["score"]
matches["w_svpt"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
804    70.0
805    94.0
806    56.0
807    68.0
808    70.0
Name: w_svpt, Length: 181114, dtype: float64

In [ ]:
doubles.dropna(axis=1, how="all")

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner1_id,winner2_id,winner_seed,...,loser2_ht,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank
2000-301,Auckland,Hard,32.000000,A,20000110,300.0,101704,101097.000000,1.0,NaN,...,USA,31.137577,13.0,0.0,15.0,0.0,16.0,0.0,17.0,0.0
2000-301,Auckland,Hard,32.000000,A,20000110,299.0,101704,101097.000000,1.0,NaN,...,CZE,28.867899,13.0,0.0,15.0,0.0,34.0,0.0,29.0,0.0
2000-301,Auckland,Hard,32.000000,A,20000110,298.0,101358,101543.000000,2.0,NaN,...,CZE,29.489391,16.0,0.0,17.0,0.0,97.0,0.0,74.0,0.0
2000-301,Auckland,Hard,32.000000,A,20000110,297.0,101704,101097.000000,1.0,NaN,...,USA,27.019849,13.0,0.0,15.0,0.0,65.0,0.0,248.0,0.0
2000-301,Auckland,Hard,32.000000,A,20000110,296.0,101358,101543.000000,2.0,NaN,...,BRA,26.297057,16.0,0.0,17.0,0.0,779.0,0.0,114.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2020-M-DC-2020-WG2-PO-MAR-VIE-01, Davis Cup WG2 PO: MAR vs VIE, Clay, 4, D, 20200306, 3, 104917, 104467, nan, nan, 109042, 122533, nan, nan, 6-4 6-1, 3, RR, Anas Fattar, R)",NaN,MAR,32.807666,Lamine Ouahab,R,185.0,ALG,35.192334,Quoc Khanh Le,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2020-M-DC-2020-WG2-PO-PAR-SRI-01, Davis Cup WG2 PO: PAR vs SRI, Clay, 4, D, 20200306, 3, 134677, 137018, nan, nan, 202462, 134886, nan, nan, 6-1 6-4, 3, RR, Juan Borba, R)",NaN,PAR,23.887748,Ayed Zatar,R,NaN,PAR,23.926078,Yasitha De Silva,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2020-M-DC-2020-WG2-PO-PHI-GRE-01, Davis Cup WG2 PO: PHI vs GRE, Clay, 4, D, 20200306, 3, 108717, 104604, nan, nan, 109737, 202065, nan, nan, 7-6(5) 6-4, 3, RR, Francis Casey Alcantara, U)",NaN,PHI,28.073922,Ruben Gonzales,R,NaN,PHI,34.472279,Markos Kalovelonis,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2020-M-DC-2020-WG2-PO-POL-HKG-01, Davis Cup WG2 PO: POL vs HKG, Hard, 4, D, 20200306, 3, 126166, 126591, nan, nan, 205695, 106388, nan, nan, 6-3 6-7(6) 6-3, 3, RR, Szymon Walkow, R)",NaN,POL,24.443532,Jan Zielinski,R,NaN,POL,23.290897,Ching Lam,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
set(matches["winner_ioc"].unique().tolist() + players["ioc"].unique().tolist())

{'AFG',
 'AHO',
 'ALB',
 'ALG',
 'AND',
 'ANG',
 'ANT',
 'ANZ',
 'ARG',
 'ARM',
 'ARU',
 'ASA',
 'AUS',
 'AUT',
 'AZE',
 'BAH',
 'BAN',
 'BAR',
 'BDI',
 'BEL',
 'BEN',
 'BER',
 'BIH',
 'BIZ',
 'BLR',
 'BOL',
 'BOT',
 'BRA',
 'BRI',
 'BRN',
 'BRU',
 'BUL',
 'BUR',
 'CAF',
 'CAL',
 'CAM',
 'CAN',
 'CAR',
 'CAY',
 'CGO',
 'CHI',
 'CHL',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COK',
 'COL',
 'CPV',
 'CRC',
 'CRO',
 'CUB',
 'CUW',
 'CYP',
 'CZE',
 'DEN',
 'DOM',
 'ECA',
 'ECU',
 'EGY',
 'ERI',
 'ESA',
 'ESP',
 'EST',
 'ETH',
 'FIJ',
 'FIN',
 'FRA',
 'FRG',
 'GAB',
 'GBR',
 'GEO',
 'GER',
 'GHA',
 'GRC',
 'GRE',
 'GRN',
 'GTM',
 'GUA',
 'GUD',
 'GUM',
 'HAI',
 'HAW',
 'HKG',
 'HON',
 'HUN',
 'INA',
 'IND',
 'IRI',
 'IRL',
 'IRQ',
 'ISL',
 'ISR',
 'ISV',
 'ITA',
 'ITF',
 'JAM',
 'JOR',
 'JPN',
 'KAZ',
 'KEN',
 'KGZ',
 'KOR',
 'KSA',
 'KUW',
 'LAO',
 'LAT',
 'LBA',
 'LBN',
 'LBR',
 'LCA',
 'LES',
 'LIB',
 'LIE',
 'LTU',
 'LUX',
 'MAD',
 'MAR',
 'MAS',
 'MDA',
 'MDG',
 'MEX',
 'MGL',
 'MHL',
 'MKD',


In [ ]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
players

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122,USA,NaN,Q54544
1,100002,Pancho,Segura,R,19210620,ECU,NaN,Q54581
2,100003,Frank,Sedgman,R,19271002,AUS,NaN,Q962049
3,100004,Giuseppe,Merlo,R,19271011,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509,USA,188.0,Q53554
...,...,...,...,...,...,...,...,...
55535,210503,Mateo,Bejenaru,U,NaN,ROU,NaN,NaN
55536,210504,Billy,Blaydes,U,NaN,GBR,NaN,NaN
55537,210505,Marcel,Zielinski,U,NaN,GER,NaN,NaN
55538,210506,Alex,Michelsen,U,NaN,USA,NaN,NaN


In [ ]:
pd.unique(matches["tourney_name"])

array(['Dublin', 'Buenos Aires', 'London', ...,
       'Davis Cup WG2 PO: EGY vs CYP', 'Davis Cup WG2 PO: GRE vs JAM',
       'Davis Cup WG2 PO: HKG vs BEN'], dtype=object)

In [ ]:
test = matches[matches["tourney_name"].str.contains("Open")] #wim, us open, french open, australian open
test[test["l_1stIn"].notnull()]
test

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
836,1968-560,US Open,Grass,96,G,19680829,1,100087,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
837,1968-560,US Open,Grass,96,G,19680829,2,100023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
838,1968-560,US Open,Grass,96,G,19680829,3,109816,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
839,1968-560,US Open,Grass,96,G,19680829,4,109932,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
840,1968-560,US Open,Grass,96,G,19680829,5,100060,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2022-580,Australian Open,Hard,128,G,20220117,222,126774,4,NaN,...,42.0,30.0,13.0,13.0,0.0,4.0,4.0,6540.0,10.0,3390.0
285,2022-580,Australian Open,Hard,128,G,20220117,223,106421,2,NaN,...,116.0,91.0,25.0,27.0,2.0,5.0,2.0,8935.0,9.0,3608.0
286,2022-580,Australian Open,Hard,128,G,20220117,224,104745,6,NaN,...,68.0,50.0,15.0,17.0,4.0,8.0,5.0,4875.0,7.0,4568.0
287,2022-580,Australian Open,Hard,128,G,20220117,225,106421,2,NaN,...,83.0,56.0,23.0,19.0,8.0,12.0,2.0,8935.0,4.0,6540.0


In [ ]:
rankings

,ranking_date,rank,player,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0
2,20000110,3,101948,2419.0
3,20000110,4,103017,2184.0
4,20000110,5,102856,2169.0
...,...,...,...,...
725601,19991227,1298,125857,1.0
725602,19991227,1298,190042,1.0
725603,19991227,1298,209471,1.0
725604,19991227,1298,209475,1.0


In [ ]:
from datetime import datetime, date

In [ ]:
def custom_filter(x):
  try:
    a = pd.to_datetime(x)
  except:
    a = None
  return a

def age(born):
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

dobs = players["dob"].apply(lambda x: custom_filter(x))
ages = dobs.apply(lambda x: age(x))
ages

0        108.0
1        100.0
2         94.0
3         94.0
4         93.0
         ...  
55535      NaN
55536      NaN
55537      NaN
55538      NaN
55539     18.0
Name: dob, Length: 55540, dtype: float64

In [ ]:
matches[matches["tourney_id"] == "2022-580"]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
162,2022-580,Australian Open,Hard,128,G,20220117,100,200175,NaN,NaN,...,41.0,25.0,12.0,12.0,7.0,15.0,77.0,836.0,146.0,458.0
163,2022-580,Australian Open,Hard,128,G,20220117,101,126205,NaN,NaN,...,56.0,33.0,12.0,13.0,5.0,10.0,41.0,1317.0,180.0,362.0
164,2022-580,Australian Open,Hard,128,G,20220117,102,106214,NaN,NaN,...,42.0,23.0,21.0,13.0,6.0,13.0,96.0,748.0,190.0,348.0
165,2022-580,Australian Open,Hard,128,G,20220117,103,132283,25,NaN,...,69.0,51.0,13.0,15.0,8.0,12.0,26.0,1860.0,110.0,685.0
166,2022-580,Australian Open,Hard,128,G,20220117,104,104792,17,NaN,...,43.0,24.0,11.0,11.0,4.0,10.0,20.0,2373.0,64.0,976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2022-580,Australian Open,Hard,128,G,20220117,222,126774,4,NaN,...,42.0,30.0,13.0,13.0,0.0,4.0,4.0,6540.0,10.0,3390.0
285,2022-580,Australian Open,Hard,128,G,20220117,223,106421,2,NaN,...,116.0,91.0,25.0,27.0,2.0,5.0,2.0,8935.0,9.0,3608.0
286,2022-580,Australian Open,Hard,128,G,20220117,224,104745,6,NaN,...,68.0,50.0,15.0,17.0,4.0,8.0,5.0,4875.0,7.0,4568.0
287,2022-580,Australian Open,Hard,128,G,20220117,225,106421,2,NaN,...,83.0,56.0,23.0,19.0,8.0,12.0,2.0,8935.0,4.0,6540.0


In [ ]:
players[players["dob"].notnull()]

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122,USA,NaN,Q54544
1,100002,Pancho,Segura,R,19210620,ECU,NaN,Q54581
2,100003,Frank,Sedgman,R,19271002,AUS,NaN,Q962049
3,100004,Giuseppe,Merlo,R,19271011,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509,USA,188.0,Q53554
...,...,...,...,...,...,...,...,...
55508,210476,Enmanuel,Munoz,U,20050122,DOM,NaN,NaN
55509,210477,Artur,Kukasian,U,20040321,RUS,NaN,NaN
55510,210478,Sebastian,Gorzny,U,20040124,USA,NaN,NaN
55511,210479,Nicolas,Eli,U,20050303,ARG,NaN,NaN


In [ ]:
doubles["tourney_date"]

2000-301                                                                                                                                                                                        300.0
2000-301                                                                                                                                                                                        299.0
2000-301                                                                                                                                                                                        298.0
2000-301                                                                                                                                                                                        297.0
2000-301                                                                                                                                                                                        296.0
          

In [ ]:
pd.unique(matches["tourney_name"][matches["tourney_name"].str.contains("Open")])

array(['US Open', 'Stockholm Open', 'Australian Open',
       'Australian Open-2', 'Us Open', 'Great Ocean Road Open',
       'Murray River Open'], dtype=object)

In [ ]:
ameteurs.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
matches.dropna(axis=1).sort_values("match_num")

,tourney_id,tourney_name,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,loser_id,loser_name,best_of,round
3280,2001-D075,Davis Cup WG PO: SVK vs CHI,4,D,20010921,1,102701,Marcelo Rios,102344,Karol Kucera,5,RR
0,1976-1725,Pepsi Grand Slam,4,A,19760710,1,100119,Ilie Nastase,100174,Manuel Orantes,3,F
982,2007-404,Indian Wells Masters,96,M,20070305,1,103084,Guillermo Canas,104225,Jan Hajek,3,R128
1629,1988-404,Indian Wells,56,M,19880229,1,101131,Jimmy Brown,101026,Jim Grabb,3,R64
3143,2005-D055,Davis Cup G2 R1: LAT vs GRE,4,D,20050304,1,104557,Andis Juska,103077,Konstantinos Economidis,5,RR
...,...,...,...,...,...,...,...,...,...,...,...,...
1056,2020-520,Roland Garros,128,G,20200928,1601,104925,Novak Djokovic,126774,Stefanos Tsitsipas,5,SF
1057,2020-520,Roland Garros,128,G,20200928,1602,104745,Rafael Nadal,106043,Diego Schwartzman,5,SF
1330,2019-520,Roland Garros,128,G,20190527,1602,104745,Rafael Nadal,103819,Roger Federer,5,SF
1331,2019-520,Roland Garros,128,G,20190527,1701,104745,Rafael Nadal,106233,Dominic Thiem,5,F


In [ ]:
rankings

,ranking_date,rank,player,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0
2,20000110,3,101948,2419.0
3,20000110,4,103017,2184.0
4,20000110,5,102856,2169.0
...,...,...,...,...
725601,19991227,1298,125857,1.0
725602,19991227,1298,190042,1.0
725603,19991227,1298,209471,1.0
725604,19991227,1298,209475,1.0


In [ ]:
players[players["height"].notnull()]

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
4,100005,Richard,Gonzalez,R,19280509,USA,188.0,Q53554
5,100006,Grant,Golden,R,19290821,USA,175.0,Q3115390
10,100011,Torben,Ulrich,L,19321004,DEN,180.0,Q785404
11,100012,Nicola,Pietrangeli,R,19330911,ITA,178.0,Q709909
14,100015,Sammy,Giammalva,R,19340801,USA,180.0,Q963700
...,...,...,...,...,...,...,...,...
54937,209903,Lukas,Neumayer,R,20020906,AUT,183.0,Q107641373
54952,209918,Hanwen,Li,R,20020822,CHN,180.0,Q106860558
54970,209936,Lorenzo,Claverie,R,20020928,ITA,196.0,NaN
54984,209950,Arthur,Fils,R,20040612,FRA,185.0,Q107207693


In [ ]:
players["wikidata_id"]

0          Q54544
1          Q54581
2         Q962049
3        Q1258752
4          Q53554
           ...   
55535         NaN
55536         NaN
55537         NaN
55538         NaN
55539         NaN
Name: wikidata_id, Length: 55540, dtype: object

In [ ]:
seed_url = "https://www.wikidata.org/wiki/"
urls = []
for wikidata_id in players[players["wikidata_id"].notnull()]["wikidata_id"]:
  urls.append(seed_url + wikidata_id)

In [ ]:
urls[0]

'https://www.wikidata.org/wiki/Q54544'

In [ ]:
pd.read_html(urls[0])

[  Language           Label             Description          Also known as
 0  English  Gardnar Mulloy  American tennis player  Gardnar Putnam Mulloy]

# Statistics

In [ ]:
halt here

SyntaxError: ignored

Memory size statistics for each dataframe

In [ ]:
# memory usage of each dataframe in megabytes
df_names = ["matches", "ameteurs", "doubles", "futures", "quals", "rankings", "players"]
dfs = [matches, ameteurs, doubles, futures, quals, rankings, players]

for name, df in zip(df_names, dfs):
  print("Memory usage of %s:" % name, round(sum(df.memory_usage())/1000000, 2), "MB")

Row, col count for each dataframe

In [ ]:
for name, df in zip(df_names, dfs):
  print("RowCount, ColCount for %s:" % name, df.shape)

NOTE: There are many nan rows as we are concatenating multiple dataframes with different columns across axis=1

Means of each dataframe

In [ ]:
means = [df.mean().rename(name) for name, df in zip(df_names, dfs)]
mean_table = pd.concat(means, axis=1)

In [ ]:
mean_table

Standard deviations for each dataframe

In [ ]:
stds = [df.std().rename(name) for name, df in zip(df_names, dfs)]
std_table = pd.concat(stds, axis=1)

In [ ]:
std_table

Medians for each dataframe

In [ ]:
medians = [df.median().rename(name) for name, df in zip(df_names, dfs)]
median_table = pd.concat(medians, axis=1)

In [ ]:
median_table

# addressing Ryan's tourney Q

In [41]:
Players = pd.read_csv("players.csv")
Tourneys = pd.read_csv("tourneys.csv")

In [47]:
Players["fullname"] = Players.agg(lambda x: f"{x['name_first']} {x['name_last']}", axis=1)

In [62]:
Tourneys["champ_id"] = Tourneys.merge(Players[["fullname", "player_id"]], left_on="champion", right_on="fullname")["player_id"]
Tourneys["runner_id"] = Tourneys.merge(Players[["fullname", "player_id"]], left_on="runner_up", right_on="fullname")["player_id"]

Tourneys["champ_id"] = Tourneys["champ_id"].fillna(-1)
Tourneys["runner_id"] = Tourneys["runner_id"].fillna(-1)

Tourneys["champ_id"] = Tourneys["champ_id"].astype(int)
Tourneys["runner_id"] = Tourneys["runner_id"].astype(int)

In [64]:
Tourneys.to_csv("tourneys.csv")

# Query Testing

first upload csv files to colab

In [16]:
Players = pd.read_csv("players.csv")
Matches = pd.read_csv("matches.csv")
Tourneys = pd.read_csv("tourneys.csv")
Rankings = pd.read_csv("rankings.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


executing query

In [14]:
q = """WITH top_five_tourneys (tourney_name) AS
   (SELECT tourney_name
    FROM matches
    GROUP BY tourney_name
    ORDER BY COUNT(*) DESC
    LIMIT 5
   ),
WITH t_five_data (tourney_name, winner_id) AS
   (SELECT tourney_name, winner_id
    FROM matches M
       INNER JOIN (SELECT * FROM top_five_tourneys) t_five ON M.tourney_name = t_five.tourney_name
    WHERE round = "F"
   ),
WITH t_five_players (player_id, height) AS
   (SELECT player_id, height
    FROM players P
       INNER JOIN t_five_data ON P.player_id = t_five_data.winner_id
   )
SELECT SUM(CASE hand WHEN 'L' THEN 1 ELSE 0 end)  * 1.0 / COUNT(*) AS l_prop,
      SUM(CASE hand WHEN 'R' THEN 1 ELSE 0 end)  * 1.0 / COUNT(*) AS r_prop,
      AVG(height) AS avg_height
      FROM t_five_players
"""

ps.sqldf(q, locals())

PandaSQLException: ignored

In [9]:
q = """SELECT surface, COUNT(*) AS frequency
FROM matches
GROUP BY surface
ORDER BY COUNT(*) DESC
"""

ps.sqldf(q, locals())

,surface,frequency
0,Hard,72505
1,Clay,65285
2,Grass,20714
3,Carpet,20324
4,None,2439


In [12]:
q = """SELECT DISTINCT name_first, name_last, earnings
FROM players P
   INNER JOIN (SELECT tourney_name, SUM(first_prize) AS earnings, champion_f_name, champion_l_name
         FROM tourneys
         GROUP BY tourney_name) T ON P.name_first = T.champion_f_name AND P.name_last = T.champion_l_name
ORDER BY earnings DESC
"""

ps.sqldf(q, locals())

,name_first,name_last,earnings
0,John,Newcombe,54174803
1,John,Newcombe,53527000


In [11]:
q = """SELECT ioc AS country, COUNT(*) AS num_champs
FROM matches M
    JOIN (SELECT player_id, ioc
          FROM players) P ON M.winner_id = P.player_id
GROUP BY country
ORDER BY COUNT(*) DESC
"""

ps.sqldf(q, locals())

,country,num_champs
0,USA,37213
1,ESP,15256
2,AUS,13772
3,FRA,11549
4,GER,8885
...,...,...
124,UAE,2
125,GUM,1
126,MNE,1
127,RHO,1
